# OpenAI - Resume & Cover Letter Pipeline
### A project that leverages OpenAI's API to enhance the job application process, crafting tailored resumes and personalized cover letters for targeted job opportunities.

In [ ]:
# Imports
import os
import webbrowser
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import tiktoken

##### Functions

In [4]:
def get_user_inputs(questions_dict):
    """Prompt the user for input based on questions provided in a dictionary.
    Allows multiline inputs and doesn't accept empty inputs.
    
    Params:
        questions_dict (dict): Keys are variable names and values are questions.
    Returns:
        user_answers (dict): Keys are variable names and values are user inputs.
    """
    user_answers = {}

    for variable, question in questions_dict.items():
        print(question)
        print("     Enter 'END' on a new line when you are finished:")

        while True:
            user_input = ""
            while True:
                line = input()
                if line.strip().upper() == 'END':  # Using 'END' as a signal to finish the input
                    break
                user_input += line + "\n"

            user_input = user_input.strip()

            if user_input:  # Check if the input is non-empty
                user_answers[variable] = user_input
                break  # Exit the loop if a valid input is received
            print("Input cannot be empty. Please enter a valid response.")

    return user_answers

In [5]:
def remove_empty_lines(input_str):
    """Removes empty lines from a multi-line string.
    
    Params:
        input_str (str): The input string with possible empty or whitespace-only lines.
    Returns:
        str: A string with empty and whitespace-only lines removed
    """
    return '\n'.join(line for line in input_str.split('\n') if line.strip())

In [6]:
def remove_stopwords(text):
    """Remove stop words and non-ASCII characters from the input text.
    
    Params:
        text (str): The input string from which stop words and non-ASCII characters will be removed.
    Returns:
        filtered_text (str): A string with stop words and non-ASCII characters removed.
    """
    stop_words = set(stopwords.words("english"))
    words = word_tokenize(text)
    # Removing non-ASCII characters
    words = [word.encode("ascii", "ignore").decode("ascii") for word in words]

    filtered_text = " ".join(word for word in words if word.lower() not in stop_words)
    return filtered_text

In [7]:
def create_prompt(questions_dict, prompts_list):
    """Create prompt (input) to use with ChatGPT.

    Params:
        questions_dict (dict): key - variable name; value - question used to get user input
        prompts_list (list of str): List of templates for the prompts
    Returns:
        prompt (str): Formatted prompt including user's inputs
    """
    # Get user input
    user_answers = get_user_inputs(questions_dict)

    # Update prompt with user input
    for prompt in prompts_list:
        try:
            prompt = prompt.format(**user_answers)
        except KeyError as e:
            print(f"Error: Missing value for placeholder {{{e.args[0]}}} in the prompt.")
            return None  # or you could return a default prompt or an error prompt

        # Remove empty lines of string
        return remove_stopwords(remove_empty_lines(prompt))

In [8]:
def count_tokens(prompt_input, encoding):
    """
    Counts tokens in the input, which can be a string, list of strings, or list of dictionaries.

    Params:
        prompt_input (str or list or dict): The input to count tokens from.
        encoding (class): tiktoken.core.Encoding -- encoding for an OpenAI model
    Returns:
        num_tokens (int): The total count of tokens.
    """
    num_tokens = 0

    def tokenize_text(text, encoding):
        """ Tokenize a given text.
        
        Params:
            text (str): The text input to process.
            encoding (class): tiktoken.core.Encoding -- encoding for an OpenAI model
        Returns:
            (int): The number of tokens in the text.
        """
        # Tokenize the text
        tokens_int = encoding.encode(text)
        # View the corresponding token string for each integer token
        print("Token String-Integer Pairs:\n",
              [encoding.decode_single_token_bytes(token) for token in tokens_int])
        return len(tokens_int)

    # Check if prompt_input is string
    if isinstance(prompt_input, str):
        #print(f"Handled a string: {input}")        
        num_tokens += tokenize_text(prompt_input, encoding) + 3 # (+3): every reply is primed with <|start|>assistant<|message|>
        return num_tokens

    # Check if prompt_input is list
    if isinstance(prompt_input, list):
        # Check if the list contains strings
        if all(isinstance(i, str) for i in prompt_input):
            #print(f"Handled a list of strings: {input}")
            for text in prompt_input:
                num_tokens += tokenize_text(text, encoding)
            num_tokens += 3
            return num_tokens

        # Check if list contains dictionaries
        if (all(isinstance(i, dict) for i in prompt_input) and
            all(all(isinstance(val, str) for val in i.values()) for i in prompt_input)):
            #print(f"Handled a list of dictionaries of strings: {input}")

            num_tokens += 4  # (+4): every message follows <|start|>{role/name}\n{content}<|end|>\n
            for message in prompt_input:
                for key, value in message.items():
                    num_tokens += tokenize_text(prompt_input, encoding)
                num_tokens += 3
            return num_tokens

        if prompt_input is None:
            return print(prompt_input)
        return "Invalid input: List must contain either strings or dictionaries of strings."
    return "Invalid input type: Must be string, list of strings, or list of dictionaries."

In [9]:
def main(model_encoders):
    """
    Counts the number of tokens in a text | message to be sent to an OpenAI model using the API

    Params:
        model_encoders (list of str): List of specific OpenAI models to be used for encoding
    Returns:
        None
    """
    # Define prompt input
    prompt = create_prompt(user_input_questions, prompts)

    # Calculate tokens required by each model encoder
    for model_encoder in model_encoders:
        encoding = tiktoken.encoding_for_model(model_encoder)
        num_tokens = count_tokens(prompt, encoding)
        print(f"\n\nThis encoding takes a total of {num_tokens} tokens (model: {model_encoder})\n")
        assert encoding.decode(encoding.encode(prompt)) == prompt, "Error: failed input encoding."


##### Parameters

In [2]:
# User input questions
user_input_questions = {
    "career": "What is your current profession or career field? ",
    "yrs_experience": "How many years of professional experience do you have? ",
    "job_posting_title": "What is the job position title you are interested in applying for? ",
    "job_description": "Please paste the entire job description into the terminal: ",
    "about_page": "Please paste the entire about page into the terminal: ",
    "resume": "Please paste your entire resume into the terminal: ",
    "eg_output": "Please paste an output example into the terminal: " # for calculating tokens
    }

In [3]:
# ChatGPT prompts
prompts = [
'''I have {yrs_experience} years of {career} experience from various projects and recently found a {job_posting_title} job posting on LinkedIn.
Your task:
    1. Review the job description, company's about page, and my resume.
    2. Craft a 1-page cover letter for the application, utilizing the provided texts.
    3. Tailor my resume to align more closely with the company and position, leveraging my experience and strengths.
    4. Return the data in a JSON format with keys "cover_leter" and  "resume", each with values of one string that contains the outputs of steps 2 and 3.
Shall we begin?
The following is the job posting:\n{job_description}
The following is the company's about page:\n{about_page}
The following is my current resume:\n{resume}\n
Complete now the above tasks 1-4.
{eg_output}'''
]

In [10]:
# OpenAI models available through the API
model_encoders = ["gpt-4",                  # for gpt-4
                "gpt-3.5-turbo",            # for gpt-3.5-turbo  
                "text-embedding-ada-002"]   # for text-embedding-ada-002
model_encoders = ["gpt-4"]  # shorter list for testing

##### Data

In [17]:
# Job Description
webbrowser.open('https://www.transcom.com/hr/careers/job-openings/data-engineer/')

True

In [18]:
# About Page
webbrowser.open('https://www.transcom.com/about-us/who-we-are/')

True

In [15]:
# Resume
os.startfile(r"C:\Users\kiahj\OneDrive\Desktop\Luminum ICT - CV ENG - Kiah Jane Jones - Transcom.docx")

In [19]:
# Output
webbrowser.open('https://chat.openai.com/c/adcf4fab-ea99-461d-8112-2a730bb1a45a')

True

##### Run

In [11]:
if __name__ == "__main__":
    main(model_encoders)

What is your current profession or career field? 
     Enter 'END' on a new line when you are finished:


 Data Science
 END


How many years of professional experience do you have? 
     Enter 'END' on a new line when you are finished:


 2
 END


What is the job position title you are interested in applying for? 
     Enter 'END' on a new line when you are finished:


...

Please paste your entire resume into the terminal: 
     Enter 'END' on a new line when you are finished:


 Kiah Jane Jones Junior Software Engineer | Data Scientist  Professional Summary Kiah Jane Jones is a devoted software engineer and data scientist embodying a blend of organizational brilliance and analytical prowess. Her code and working style are reflections of meticulous organization, ensuring a smooth, efficient workflow and clear, maintainable code. Kiah's approach is both analytical and creative, allowing for innovative solutions that are both logically sound and outside the conventional box.  In addition to her technical acumen, Kiah brings strong collaborative and communication skills to the table, fortified by proficiency in several European languages. This multicultural linguistic ability enhances her capacity to thrive in diverse team environments, facilitating effective and inclusive communication strategies. Armed with these qualities, Kiah is primed to contribute innovative data engineering solutions and vibrant team dynamics to Transcom, enhancing project outcomes and te

Please paste an output example into the terminal: 
     Enter 'END' on a new line when you are finished:


 Perfect, thank you for sharing your resume. Based on the information provided, I will craft a cover letter and then tailor your resume to align better with the Data Engineer position at Transcom.  ---  ### Cover Letter  [Your Address]   [City, State, Zip]   [Your Email]   [Your Phone Number]   [Date]  Hiring Manager   Transcom   [Company’s Address]   [City, State, Zip]  Dear Hiring Manager,  I am writing to express my interest in the Data Engineer position at Transcom, a renowned global customer service provider known for its relentless commitment to innovation and excellence. With my background as a Data Scientist and Software Engineer, accompanied by a rich blend of skills in data lifecycle management, machine learning, and software development, I am enthusiastic about the opportunity to contribute to Transcom's mission of delivering smarter people experiences through cutting-edge technological solutions.  Your commitment to driving brands forward with digital excellence and innovat

TypeError: count_tokens.<locals>.tokenize_text() missing 1 required positional argument: 'encoding'